In [1]:
library(dplyr)
library(ggplot2)
library(corrplot)

#### DATA LOADING, CLEANING AND SOME EXPLORATORY ANALYSIS ####

# more info on dataset can be found at https://archive.ics.uci.edu/dataset/1/abalone
aburl = 'http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
abalone = read.table(aburl, header = F , sep = ',')
 

head(abalone) # always good to get a glimpse

#V1,V2, ... not really informative as variable names.
colnames(abalone)<- c('sex','length','diameter','height','weight.w',
                      'weight.s','weight.v','weight.sh','rings')
head(abalone) # much better

# inspired by info on the dataset let's add age as a variable
abalone$age <- abalone$rings + 1.5

# shows descriptive statistics for each variable included
summary(abalone) # clearly variable SEX is not correctly interpreted

# proper way to encode categorical variables
abalone$sex <- as.factor(abalone$sex)
summary(abalone$sex) # now it's good

# in this toy dataset it is not the case but one crucial, 
# annoying yet essential step is data-cleaning. Specific steps depend on the particular dataset but
# some general guidelines:
#     - check for missing values (NA, NaN, "") -> either remove or fill in some way
#     - check for values that don't make sense given the covariate's meaning (e.g. negative heights) -> either remove or correct in some way
#     - check for Inf values -> depending on the meaning it might make sense to remove or transform the covariate in some way that results in finite values


# effective plotting to get a sense of what the data looks like and inform on the relations between variables is essential
# plot(abalone) # can get very confusing very fast as the number of variables increases


# let's have a look at how length is distributed according to sex
# boxplots are useful in this case
boxplot(abalone$length ~ abalone$sex, xlab = "Sex", ylab = "Length")
lines(seq(0.5,1.5),rep(mean(abalone$length[abalone$sex=="F"]),2), 
      lwd= 1.5, col = "darkorange")
lines(seq(1.5,2.5),rep(mean(abalone$length[abalone$sex=="I"]),2), 
      lwd= 1.5, col = "darkorange")
lines(seq(2.5,3.5),rep(mean(abalone$length[abalone$sex=="M"]),2), 
      lwd= 1.5, col = "darkorange")

# alternatively histograms help too:
ggplot(abalone,aes(length))+
  geom_histogram(binwidth = 0.025)+
  facet_wrap(~sex)+
  ggtitle("Histogram of Length by Sex")

# or perhaps density plots
ggplot(abalone,aes(length,col=sex,group=sex))+
  geom_density()+
  ggtitle("Density of Length by Sex")

# or barplots
abalone %>% 
  group_by(sex) %>% 
  summarise(length_mean=mean(length)) %>% 
  ggplot(aes(sex,length_mean))+
  geom_bar(stat="identity")+
  labs(y = "average length")+
  ggtitle("Average length by sex")

# to catch a glimpse of relation between continuous variables 
# let's look at the correlation matrix 
# REMEMBER: correlation between 2 random variables measures the strength of linear association between the 2
# with positive or negative values suggesting resp. an increasing or decreasing trend
corr_matrix = cor(abalone[,-1])
corrplot(corr_matrix, method = 'number', type = 'lower', diag = FALSE
         , col = COL2('BrBG'), tl.col = "black")

# but first plots suggested that not all pairwise relations
# between variables make sense as linear

# let's look at relation between length and weight.w
ggplot(data=abalone,
       aes(x=length,y=weight.w)) +
  geom_point(alpha=0.5)+
  ggtitle('Abalone whole weight by length')

# separating by Sex might also be useful
ggplot(abalone,aes(length,weight.w))+ # prepare axes
  geom_point( color = "darkorange")+  # add dots
  stat_smooth(method='lm',se=FALSE) + # se = standard error
  facet_wrap(~sex) +  # create different plots by sex groups
  ggtitle("Abalone weight by length and sex")
# even within individual groups, the non-linearity  between weight.w 
# and length is still evident. 

# On occasion, it’s beneficial to construct artificial groups. 
# For example, we can create a binary variable for length (length_bin) 
# with values “short” (below average) and “long” (above average).
abalone$length_bin = as.factor(ifelse(abalone$length<mean(abalone$length),
                            "short","long"))
summary(abalone$length_bin)

ggplot(abalone,aes(length_bin,weight.w))+
  geom_boxplot()+
  ggtitle("Abalone weight by length_bin")

ggplot(abalone,aes(length_bin,weight.w))+
  geom_boxplot()+
  ggtitle("Abalone weight by length_bin")+
  facet_wrap(~sex)

# by looking at this plot what can we say about whether there is a difference in the distribution 
# of the weight between the groups identified by long/short length ??
# not too much statistically speaking but hypothesis testing can help 
# (comes in a few lines of code)

############ SOME STATISTICAL ANALYSIS ############

####### LINEAR MODELLING #######

# let's try this: say we are interested in understanding relation between diameter and length
m0 <- lm(length~diameter,data=abalone)
summary(m0)

plot(abalone$diameter,abalone$length, xlab = "Diameter", ylab = "Length", col = "darkorange")
abline(m0$coefficients[1], m0$coefficients[2], col = "blue", lwd = 2, lty = 2)

# maybe height is relevant too ? 
m1 <- lm(length~diameter + height,data=abalone)
summary(m1)

# what happens if we add weight ? Something interesting changes for height
m2 <- lm(length~diameter + height + weight.w, data=abalone)
summary(m2)

# so let's try this one
m3 <- lm(length~diameter + weight.w, data=abalone)
summary(m3)


# model comparison according to AIC and BIC
cat("Model ", which.min(c(AIC(m0),AIC(m1),AIC(m2),AIC(m3))) -1, "is selected according to AIC. \n")
cat("Model ", which.min(c(BIC(m0),BIC(m1),BIC(m2),BIC(m3))) -1, "is selected according to BIC. \n")
print(formula(m3)) # Notice something interesting ??

# let's us now start from weight.w instead
m0 <- lm(length~weight.w,data=abalone)
summary(m0)

plot(abalone$weight.w,abalone$length, xlab = "Weight", ylab = "Length", col = "darkorange")
abline(m0$coefficients[1], m0$coefficients[2], col = "blue", lwd = 2, lty = 2)
# not surprising we saw earlier that the relation between the two was not linear.

# let's try with a transformation of weight.w
m0_tr <- lm(length~sqrt(weight.w),data=abalone)
summary(m0_tr)

plot(sqrt(abalone$weight.w),abalone$length, xlab = "SQRT(Weight)", ylab = "Length", col = "darkorange")
abline(m0_tr$coefficients[1], m0_tr$coefficients[2], col = "blue", lwd = 2, lty = 2)

# maybe diameter is relevant too ? 
m1_tr <- lm(length~sqrt(weight.w)+ diameter,data=abalone)
summary(m1_tr)

# what happens if we add height ? 
m2_tr <- lm(length~diameter + height + sqrt(weight.w), data=abalone)
summary(m2_tr)

cat("Model ", which.min(c(AIC(m0_tr),AIC(m1_tr),AIC(m2_tr))) -1, "is selected according to AIC. \n")
cat("Model ", which.min(c(BIC(m0_tr),BIC(m1_tr),BIC(m2_tr))) -1, "is selected according to BIC. \n")
print(formula(m2))

# If we compare these set of models with the transformed weight.w variable to the previous ones
# where it is not transformed,  which is the best among these ones? The one with or without height?
# the one with the transformed weight.w variables ?? DO IT AND FIND OUT....


### let's now move on to age:

# what about this ? 
m0_bis <- lm(age ~ rings,data=abalone)
summary(m0_bis) # kind of a useless fit given the definition of age = rings + 1.5
plot(abalone$rings,abalone$age, xlab = "Rings", ylab = "Age", col = "darkorange")
abline(m0_bis$coefficients[1], m0_bis$coefficients[2], col = "blue", lwd = 2, lty = 2)

# let's focus on age and more sensible predictors: indeed the objective is to avoid having
# to count rings...

# let's put all variables in (except rings - we know why - and sex and length_bin
# - only because for now we are avoiding categorical variables -).
abalone_msr <- abalone %>% select(-c(sex,rings, length_bin) )

m_age_allmsr <- lm(age~. , data=abalone_msr) # no transformation applied to any of the predictors here...
summary(m_age_allmsr)  
# length is not significant 
# let's remove it and compare with the other models
summary(lm(age~. - length , data=abalone_msr))


# on the opposite extreme let's take this one
m_age0 <-lm(age~weight.sh,data=abalone)
summary(m_age0)
plot(abalone$weight.sh,abalone$age, xlab = "Shell weight", ylab = "Age", col = "darkorange")
abline(m_age0$coefficients[1], m_age0$coefficients[2], col = "blue", lwd = 2, lty = 2) # not great

# what if I add diameter
m_age1 <-lm(age~weight.sh + diameter,data=abalone)
summary(m_age1)

# or length
m_age2 <-lm(age ~ weight.sh + length,data=abalone)
summary(m_age2)

# or both
m_age3 <-lm(age ~ weight.sh + length + diameter,data=abalone)
summary(m_age3)

# and height 
m_age4 <-lm(age ~ weight.sh + length + diameter + height,data=abalone)
summary(m_age4)

# let's compare using AIC and BIC
mods_age<-list(m_age_allmsr, m_age0,m_age1,m_age2,m_age3, m_age4)
AIC_mods<-c(AIC(m_age_allmsr),AIC(m_age0),AIC(m_age1),AIC(m_age2),AIC(m_age3),AIC(m_age4))
BIC_mods<-c(BIC(m_age_allmsr),BIC(m_age0),BIC(m_age1),BIC(m_age2),BIC(m_age3),BIC(m_age4))

best_mod_age_AIC <- formula(mods_age[[which.min(AIC_mods)]])
best_mod_age_BIC <- formula(mods_age[[which.min(BIC_mods)]])
cat("Model",paste(deparse(best_mod_age_AIC)), "is selected according to AIC. \n")
cat("Model ", paste(deparse(best_mod_age_BIC)), "is selected according to BIC. \n")


# Let's try again from the one covariates model but
# with a transformation to see how it works
m_age0_sqrt <-lm(age~sqrt(weight.sh),data=abalone)
summary(m_age0_sqrt)
plot(sqrt(abalone$weight.s),abalone$age, xlab = "SQRT(Shell weight)", ylab = "Age", col = "darkorange")
abline(m_age0_sqrt$coefficients[1], m_age0_sqrt$coefficients[2], col = "blue", lwd = 2, lty = 2) 
# a little better ? Let's check with AIC and BIC

AIC(m_age0)
AIC(m_age0_sqrt)

m_age1d <-lm(age~sqrt(weight.sh) + diameter,data=abalone)
summary(m_age1d) # diameter significant

m_age1l <-lm(age~sqrt(weight.sh) + length,data=abalone)
summary(m_age1l)  # length  significant

m_age1h <-lm(age~sqrt(weight.sh) + height, data=abalone)
summary(m_age1h) # height is not

# what if we add both length and diameter ?
m_age1dh <-lm(age~sqrt(weight.sh) + length +  diameter , data=abalone)
summary(m_age1dh)

# what if we add all 3 of them plus sqrt(weight.sh)  ?
m_age1dlh <-lm(age~sqrt(weight.sh) + height +  length + diameter , data=abalone)
summary(m_age1dlh)

mods_age<-list(m_age1d, m_age1l,m_age1h,m_age1dh,m_age1dlh)
AIC_mods<-c(AIC(m_age1d),AIC(m_age1l),AIC(m_age1h),AIC(m_age1dh),AIC(m_age1dlh))
BIC_mods<-c(BIC(m_age1d),BIC(m_age1l),BIC(m_age1h),BIC(m_age1dh),BIC(m_age1dlh))

best_mod_age_AIC <- formula(mods_age[[which.min(AIC_mods)]])
best_mod_age_BIC <- formula(mods_age[[which.min(BIC_mods)]])
cat("Model",paste(deparse(best_mod_age_AIC)), "is selected according to AIC. \n")
cat("Model ", paste(deparse(best_mod_age_BIC)), "is selected according to BIC. \n")
# two different models are selected... 

# what if we transformed weight using log instead ? TRY IT OUT

######## A DIFFERENT KIND OF T-TEST :

# to determine if there’s a statisticallysignificant difference between the two groups (identified 
# by a binary variable)
# we can conduct a t-test for the difference of the mean between the groups

# in our case we are comparing mean for the variable weight.w between groups "long" and
# "short" for length 
t.test(abalone$weight.w~abalone$length_bin)
# pvalue < alpha even for very small values of alpha
# we reject H0, which means data suggests a difference between the means of the groups


# let's try to do something similar but with difference of the length distribution
# between groups defined by Sex = F and Sex = M (yes technically we have 3 categories,
# but to simplify we momentarily assume that it is not the case)

# selecting portion of dataframe that involves only Sex = F or M
rows_FM<-which(abalone$sex == "F" | abalone$sex == "M")
red_FM_abalone<- abalone[rows_FM,]
red_FM_abalone$sex <-as.factor(as.character(red_FM_abalone$sex))
t.test(red_FM_abalone$weight.w~red_FM_abalone$sex)
# by looking at the pvalue what can we conclude wrt evidence for H0 (mean_length_F = mean_length_M ) 
# from the data ?


########### NEXT: inclusion of categorical varibales (in this instance sex) into the regression
# and reasoning on appropriate interpretation of the coefficients estimates.



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


corrplot 0.92 loaded

Warning message in file(file, "rt"):
“URL 'http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data': status was 'Couldn't resolve host name'”


ERROR: Error in file(file, "rt"): cannot open the connection to 'http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
